In [2]:
pip install transformers datasets torch sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [2]:
from datasets import load_dataset

# Load dataset (assuming you have a .txt file with conversation pairs)
dataset = load_dataset('text', data_files={'train': r'C:\Users\ayush\Downloads\museum_ticket_data.txt'})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [7]:
import torch
# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set model to training mode
model.train()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(1):  # You can adjust the number of epochs
    for batch in data:
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        
        # Get the loss
        loss = outputs.loss
        print("Loss:", loss.item())
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


NameError: name 'data' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

# Fine-tune the model
trainer.train()


In [ ]:
from googletrans import Translator

translator = Translator()

def translate_text(text, dest_language='en'):
    translation = translator.translate(text, dest=dest_language)
    return translation.text

# Example usage in chatbot flow
user_input = "¿Cuántos boletos puedo comprar?"
translated_input = translate_text(user_input)
print(f"Translated: {translated_input}")


In [ ]:
import speech_recognition as sr

recognizer = sr.Recognizer()

def recognize_speech():
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

        try:
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            return "Sorry, I could not understand the audio."
        except sr.RequestError:
            return "Speech Recognition API is unavailable."

# Example usage in chatbot flow
user_input = recognize_speech()
if user_input:
    translated_input = translate_text(user_input)
    response = generate_response(translated_input)  # GPT-2 generated response
    print(f"ChatGPT: {response}")


In [ ]:
def generate_response(user_input):
    inputs = tokenizer.encode(user_input, return_tensors='pt')
    outputs = model.generate(inputs, max_length=100, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
user_input = "Book me two tickets for the National Museum."
bot_response = generate_response(user_input)
print(f"ChatGPT: {bot_response}")


In [ ]:
while True:
    # Get user input via speech recognition
    user_input = recognize_speech()
    
    # Translate to English if necessary
    translated_input = translate_text(user_input)

    # Generate GPT-2 response
    gpt2_response = generate_response(translated_input)

    # Output response
    print(f"ChatGPT: {gpt2_response}")

    if "exit" in translated_input.lower():
        break
